# 코루틴 사용하기
지금까지 함수를 호출한 뒤 함수가 끝나면 현재 코드로 다시 돌아왔습니다. 예를 들어서 다음과 같이 calc 함수 안에서 add 함수를 호출했을 때 add 함수가 끝나면 다시 calc 함수로 돌아옵니다. 특히 add 함수가 끝나면 이 함수에 들어있던 변수와 계산식은 모두 사라집니다.

In [1]:
def add(a, b):
    c = a + b    # add 함수가 끝나면 변수와 계산식은 사라짐
    print(c)
    print('add 함수')


def calc():
    add(1, 2)    # add 함수가 끝나면 다시 calc 함수로 돌아옴
    print('calc 함수')


calc()

3
add 함수
calc 함수


메인 루틴에서 서브 루틴을 호출하면 서브 루틴의 코드를 실행한 뒤 다시 메인 루틴으로 돌아옵니다. 특히 서브 루틴이 끝나면 서브 루틴의 내용은 모두 사라집니다. 즉, 서브 루틴은 메인 루틴에 종속된 관계입니다.

하지만 코루틴은 방식이 조금 다릅니다. 코루틴(coroutine)은 cooperative routine를 의미하는데 서로 협력하는 루틴이라는 뜻입니다. 즉, 메인 루틴과 서브 루틴처럼 종속된 관계가 아니라 서로 대등한 관계이며 특정 시점에 상대방의 코드를 실행합니다.

이처럼 코루틴은 함수가 종료되지 않은 상태에서 메인 루틴의 코드를 실행한 뒤 다시 돌아와서 코루틴의 코드를 실행합니다. 따라서 코루틴이 종료되지 않았으므로 코루틴의 내용도 계속 유지됩니다.

일반 함수를 호출하면 코드를 한 번만 실행할 수 있지만, 코루틴은 코드를 여러 번 실행할 수 있습니다. 참고로 함수의 코드를 실행하는 지점을 진입점(entry point)이라고 하는데, 코루틴은 진입점이 여러 개인 함수입니다.

# 코루틴에 값 보내기

코루틴은 제너레이터의 특별한 형태입니다. 제너레이터는 yield로 값을 발생시켰지만 코루틴은 yield로 값을 받아올 수 있습니다. 


다음과 같이 코루틴에 값을 보내면서 코드를 실행할 때는 send 메서드를 사용합니다. 그리고 send 메서드가 보낸 값을 받아오려면 **(yield)** 형식으로 yield를 괄호로 묶어준 뒤 변수에 저장합니다.]

- **`코루틴객체.send(값)`**
- **`변수 = (yield)`**

In [2]:
def number_coroutine():
    while True:        # 코루틴을 계속 유지하기 위해 무한 루프 사용
        x = (yield)    # 코루틴 바깥에서 값을 받아옴, yield를 괄호로 묶어야 함
        print(x)


co = number_coroutine()
next(co)      # 코루틴 안의 yield까지 코드 실행(최초 실행)

co.send(1)    # 코루틴에 숫자 1을 보냄
co.send(2)    # 코루틴에 숫자 2을 보냄
co.send(3)    # 코루틴에 숫자 3을 보냄

1
2
3


먼저 코루틴 number_coroutine은 while True:로 무한히 반복하도록 만듭니다. 왜냐하면 코루틴을 종료하지 않고 계속 유지시키기 위해 무한 루프를 사용합니다(코루틴을 종료하는 방법은 뒤에서 설명하겠습니다). 그리고 x = (yield)와 같이 코루틴 바깥에서 보낸 값을 받아서 x에 저장하고, print로 x의 값을 출력합니다.

코루틴 바깥에서는 co = number_coroutine()과 같이 코루틴 객체를 생성한 뒤 next(co)로 코루틴 안의 코드를 최초로 실행하여 yield까지 코드를 실행합니다(co.__next__()를 호출해도 상관없습니다).

그다음에 co.send로 숫자 1, 2, 3을 보내면 코루틴 안에서 숫자를 받은 뒤 print로 출력합니다.

1. 먼저 next(co)로 코루틴의 코드를 최초로 실행하면 x = (yield)의 yield에서 대기하고 다시 메인 루틴으로 돌아옵니다.

2. 그다음에 메인 루틴에서 co.send(1)로 1을 보내면 코루틴은 대기 상태에서 풀리고 x = (yield)의 x = 부분이 실행된 뒤 print(x)로 숫자를 출력합니다. 이 코루틴은 while True:로 반복하는 구조이므로 다시 x = (yield)의 yield에서 대기합니다. 그러고 나서 메인 루틴으로 돌아옵니다. 이런 과정으로 send가 보낸 값을 (yield)가 받게 됩니다.

3. 계속 같은 과정으로 send를 사용하여 값을 보내면 코루틴에서 값을 받아서 출력합니다.

정리하자면 next 함수(__next__ 메서드)로 코루틴의 코드를 최초로 실행하고, send 메서드로 코루틴에 값을 보내면서 대기하고 있던 코루틴의 코드를 다시 실행합니다.

지금까지 코루틴의 코드를 최초로 실행할 때 next 함수(__next__ 메서드)를 사용했지만, 코루틴객체.send(None)과 같이 send 메서드에 None을 지정해도 코루틴의 코드를 최초로 실행할 수 있습니다.

# 코루틴 바깥으로 값 전달하기
지금까지 코루틴 안에 값을 보내기만 했는데 이번에는 코루틴에서 바깥으로 값을 전달해보겠습니다. 다음과 같이 **(yield 변수)** 형식으로 yield에 변수를 지정한 뒤 괄호로 묶어주면 값을 받아오면서 바깥으로 값을 전달합니다. 그리고 yield를 사용하여 바깥으로 전달한 값은 next 함수(__next__ 메서드)와 send 메서드의 반환값으로 나옵니다.

- **`변수 = (yield 변수)`**
- **`변수 = next(코루틴객체)`**
- **`변수 = 코루틴객체.send(값)`**


In [3]:
def sum_coroutine():
    total = 0
    while True:
        x = (yield total)    # 코루틴 바깥에서 값을 받아오면서 바깥으로 값을 전달
        total += x


co = sum_coroutine()
print(next(co))      # 0: 코루틴 안의 yield까지 코드를 실행하고 코루틴에서 나온 값 출력

print(co.send(1))    # 1: 코루틴에 숫자 1을 보내고 코루틴에서 나온 값 출력
print(co.send(2))    # 3: 코루틴에 숫자 2를 보내고 코루틴에서 나온 값 출력
print(co.send(3))    # 6: 코루틴에 숫자 3을 보내고 코루틴에서 나온 값 출력

0
1
3
6


코루틴에서 값을 누적할 변수 total를 만들고 0을 할당합니다. 그리고 x = (yield total)과 같이 값을 받아오면서 바깥으로 값을 전달하도록 만듭니다. 즉, 바깥에서 send가 보낸 값은 x에 저장되고, 코루틴 바깥으로 보낼 값은 total입니다. 그다음에 total += x와 같이 받은 값을 누적해줍니다.

코루틴 바깥에서는 co = sum_coroutine()과 같이 코루틴 객체를 생성한 뒤 next(co)로 코루틴 안의 코드를 최초로 실행하여 yield까지 코드를 실행하고, print로 next(co)에서 반환된 값을 출력합니다. 그다음에 co.send로 숫자 1, 2, 3을 보내고, print로 co.send에서 반환된 값을 출력합니다.

참고로 next와 send의 차이를 살펴보면 next는 코루틴의 코드를 실행하지만 값을 보내지 않을 때 사용하고, send는 값을 보내면서 코루틴의 코드를 실행할 때 사용합니다.

1. 먼저 next(co)로 코루틴의 코드를 최초로 실행하면 x = (yield total)의 yield에서 total을 메인 루틴으로 전달하고 대기합니다. 그다음에 메인 루틴에서 print(next(co))와 같이 코루틴에서 나온 값을 출력합니다. 여기서는 total에 0이 들어있으므로 0을 받아와서 출력합니다.

2. 그리고 co.send(1)로 1을 보내면 코루틴은 대기 상태에서 풀리고 x = (yield total)의 x = 부분이 실행된 뒤 total += x로 숫자를 누적합니다. 이 코루틴은 while True:로 반복하는 구조이므로 다시 x = (yield total)의 yield에서 total을 메인 루틴으로 전달하고 대기합니다. 그다음에 메인 루틴에서 print(co.send(1))과 같이 코루틴에서 나온 값을 출력합니다. 여기서는 total에 1이 들어있으므로 1을 받아와서 출력합니다.

3. 이런 과정으로 (yield total)이 바깥으로 전달한 값을 next와 send의 반환값으로 받고, send가 보낸 값을 x = (yield total)의 x가 받게 됩니다.

여기서는 yield를 사용하여 코루틴 바깥으로 값을 전달하면 next와 send의 반환값으로 받는다는 점만 기억하면 됩니다.

- 제너레이터는 next 함수(__next__ 메서드)를 반복 호출하여 값을 얻어내는 방식
- 코루틴은 next 함수(__next__ 메서드)를 한 번만 호출한 뒤 send로 값을 주고 받는 방식

# 코루틴을 종료하고 예외 처리하기

## 코루틴을 종료하고 예외 처리하기
보통 코루틴은 실행 상태를 유지하기 위해 while True:를 사용해서 끝나지 않는 무한 루프로 동작합니다. 만약 코루틴을 강제로 종료하고 싶다면 close 메서드를 사용합니다.

- **`코루틴객체.close()`**

In [4]:
def number_coroutine():
    while True:
        x = (yield)
        print(x, end=' ')


co = number_coroutine()
next(co)

for i in range(20):
    co.send(i)

co.close()    # 코루틴 종료

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

코루틴 객체에서 close 메서드를 사용하면 코루틴이 종료됩니다. 

## GeneratorExit 예외 처리하기
코루틴 객체에서 close 메서드를 호출하면 코루틴이 종료될 때 GeneratorExit 예외가 발생합니다. 따라서 이 예외를 처리하면 코루틴의 종료 시점을 알 수 있습니다.

In [5]:
def number_coroutine():
    try:
        while True:
            x = (yield)
            print(x, end=' ')
    except GeneratorExit:    # 코루틴이 종료 될 때 GeneratorExit 예외 발생
        print()
        print('코루틴 종료')


co = number_coroutine()
next(co)

for i in range(20):
    co.send(i)

co.close()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
코루틴 종료


코루틴 안에서 try except로 GeneratorExit 예외가 발생하면 '코루틴 종료'가 출력되도록 만들었습니다. 이렇게 하면 close 메서드로 코루틴을 종료할 때 원하는 코드를 실행할 수 있습니다.

## 코루틴 안에서 예외 발생시키기
코루틴 안에 예외를 발생 시킬 때는 throw 메서드를 사용합니다. throw는 말그대로 던지다라는 뜻인데 예외를 코루틴 안으로 던집니다. 이때 throw 메서드에 지정한 에러 메시지는 except as의 변수에 들어갑니다.

- **`코루틴객체.throw(예외이름, 에러메시지)`**

In [6]:
def sum_coroutine():
    try:
        total = 0
        while True:
            x = (yield)
            total += x
    except RuntimeError as e:
        print(e)
        yield total    # 코루틴 바깥으로 값 전달


co = sum_coroutine()
next(co)

for i in range(20):
    co.send(i)

print(co.throw(RuntimeError, '예외로 코루틴 끝내기'))  # 190
# 코루틴의 except에서 yield로 전달받은 값

예외로 코루틴 끝내기
190


코루틴 안에서 try except로 RuntimeError 예외가 발생하면 print로 에러 메시지를 출력하고 yield를 사용하여 total을 바깥으로 전달하도록 만들었습니다.

코루틴 바깥에서는 co.throw(RuntimeError, '예외로 코루틴 끝내기')와 같이 throw 메서드에 RuntimeError 예외와 에러 메시지를 지정하면 코루틴 안에서 예외가 발생합니다. 그리고 코루틴 안의 except에서 yield를 사용하여 바깥으로 전달한 값은 throw 메서드의 반환값으로 나옵니다

# 하위 코루틴의 반환값 가져오기

## 하위 코루틴의 반환값 가져오기
제너레이터에서 yield from을 사용하면 값을 바깥으로 여러 번 전달한다고 했습니다('40.3 yield from으로 값을 여러 번 바깥으로 전달하기' 참조). 

하지만 코루틴에서는 조금 다르게 사용합니다. yield from에 코루틴를 지정하면 해당 코루틴에서 return으로 반환한 값을 가져옵니다(yield from은 파이썬 3.3 이상부터 사용 가능)

- **`변수 = yield from 코루틴()`**

In [7]:
def accumulate():
    total = 0
    while True:
        x = (yield)         # 코루틴 바깥에서 값을 받아옴
        if x is None:       # 받아온 값이 None이면
            return total    # 합계 total을 반환
        total += x


def sum_coroutine():
    while True:
        total = yield from accumulate()    # accumulate의 반환값을 가져옴
        print(total)


co = sum_coroutine()
next(co)

for i in range(1, 11):    # 1부터 10까지 반복
    co.send(i)            # 코루틴 accumulate에 숫자를 보냄
co.send(None)             # 코루틴 accumulate에 None을 보내서 숫자 누적을 끝냄

for i in range(1, 101):   # 1부터 100까지 반복
    co.send(i)            # 코루틴 accumulate에 숫자를 보냄
co.send(None)             # 코루틴 accumulate에 None을 보내서 숫자 누적을 끝냄

55
5050


코루틴에 1부터 10까지 보내서 합계 55를 구하고, 다시 1부터 100까지 보내서 합계 5050을 구했습니다.

먼저 숫자를 받아서 누적할 코루틴을 만듭니다. x = (yield)와 같이 코루틴 바깥에서 값을 받아온 뒤 total에 계속 더합니다. 특히 이 코루틴은 while True:로 무한히 반복하지만 코루틴을 끝낼 방법이 필요합니다. 여기서는 코루틴 바깥에서 받아온 값이 None이면 return으로 total을 반환하고 코루틴을 끝냅니다.

이제 합계를 출력할 코루틴을 만듭니다. 먼저 while True:로 무한히 반복합니다. 그리고 total = yield from accumulate()와 같이 yield from을 사용하여 코루틴 accumulate의 반환값을 가져옵니다.

코루틴에서 yield from을 사용하면 코루틴 바깥에서 send로 하위 코루틴까지 값을 보낼 수 있습니다. 따라서 co = sum_coroutine()으로 코루틴 객체를 만든 뒤 co.send로 값을 보내면 accumulate에서 값을 받습니다.

co.send로 숫자를 계속 보내다가 누적을 끝내고 싶으면 None을 보내면 됩니다.

이때 accumulate는 None을 받으면 코루틴이 완전히 끝나지만 sum_coroutine에서 무한 루프로 반복하고 있으므로 print로 total을 출력한 뒤 다시 yield from accumulate()로accumulate를 실행하게 됩니다.

## StopIteration 예외 발생시키기

코루틴도 제너레이터이므로 return을 사용하면 StopIteration이 발생합니다. 사실 코루틴에서 **return 값**은 **raise StopIteration(값)**과 동작이 같습니다. 따라서 raise로 예외를 직접 발생시키고 값을 지정하면 yield from으로 값을 가져올 수 있습니다.

- **`raise StopIteration(값)`**

In [11]:
def accumulate():
    total = 0
    while True:
        x = (yield)                       # 코루틴 바깥에서 값을 받아옴
        if x is None:                     # 받아온 값이 None이면
            # StopIteration에 반환할 값을 지정(파이썬 3.6 이하)
            raise StopIteration(total)
        total += x


def sum_coroutine():
    while True:
        total = yield from accumulate()    # accumulate의 반환값을 가져옴
        print(total)


co = sum_coroutine()
next(co)

for i in range(1, 11):    # 1부터 10까지 반복
    co.send(i)            # 코루틴 accumulate에 숫자를 보냄
co.send(None)             # 코루틴 accumulate에 None을 보내서 숫자 누적을 끝냄

for i in range(1, 101):   # 1부터 100까지 반복
    co.send(i)            # 코루틴 accumulate에 숫자를 보냄
co.send(None)

RuntimeError: generator raised StopIteration

accumulate에서 return total 대신 raise StopIteration(total)을 사용했습니다. 이때도 yield from은 accumulate의 total을 가져옵니다.

## 코루틴의 yield from으로 값을 발생시키기
이번 예제에서는 x = (yield)와 같이 코루틴 바깥에서 보낸 값만 받아왔습니다. 하지만 코루틴에서 yield에 값을 지정해서 바깥으로 전달했다면 yield from은 해당 값을 다시 바깥으로 전달합니다.

In [12]:
def number_coroutine():
    x = None
    while True:
        x = (yield x)    # 코루틴 바깥에서 값을 받아오면서 바깥으로 값을 전달
        if x == 3:
            return x


def print_coroutine():
    while True:
        x = yield from number_coroutine()   # 하위 코루틴의 yield에 지정된 값을 다시 바깥으로 전달
        print('print_coroutine:', x)


co = print_coroutine()
next(co)

x = co.send(1)    # number_coroutine으로 1을 보냄
print(x)          # 1: number_coroutine의 yield에서 바깥으로 전달한 값
x = co.send(2)    # number_coroutine으로 2를 보냄
print(x)          # 2: number_coroutine의 yield에서 바깥으로 전달한 값
co.send(3)        # 3을 보내서 반환값을 출력하도록 만듦

1
2
print_coroutine: 3


# 연습

## 문자열 검색 코루틴 만들기

In [13]:
def find(word):
    result = False

    while True:
        line = (yield result)
        result = word in line


f = find("Python")
next(f)

print(f.send("Hello, Python!"))
print(f.send("Hello, world!"))
print(f.send("Python Script"))

f.close()

True
False
True


## 사칙연산 코루틴 만들기

In [14]:
def calc():
    result = 0
    while True:
        expression = (yield result)
        a, operator, b = expression.split()
        if operator == "+":
            result = int(a) + int(b)
        elif operator == "-":
            result = int(a) - int(b)
        elif operator == "*":
            result = int(a) * int(b)
        elif operator == "/":
            result = int(a) / int(b)


expressions = input().split(", ")

c = calc()
next(c)

for e in expressions:
    print(c.send(e))

c.close()

 3 + 4


7
